<a href="https://colab.research.google.com/github/dmourlot89/MELI_challenge/blob/main/MELI_DSChallenge_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3. Similitud entre productos

**Descripción:**

Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “`items_titles.csv`” que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil.

El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset `item_titles_test.csv` generando como output un listado en el cual, ordenando por score de similitud, podamos encontrar los pares de productos más similares en nuestro dataset de test.

In [2]:
# import required libraries and functions
import pandas as pd              # Pandas for dataset operations
from itertools import product    # This function computes the cartesian product of input iterables.

In [3]:
# load dataset
url = 'https://raw.githubusercontent.com/dmourlot89/MELI_challenge/main/items_titles.csv'

df = pd.read_csv(url)

In [ ]:
df.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


In [4]:
def computeSimilarityScore(title1, title2):
  '''
  Computes Jaccard Index (similarity) between two sentences (titles)
  '''
  label1 = set(title1.lower().split())
  label2 = set(title2.lower().split())

  return len(label1.intersection(label2))/len(label1.union(label2))


In [ ]:
sim_df = pd.DataFrame(columns=['TITLE_1', 'TITLE_2', 'SIMILARITY_SCORE'])
rows = range(df.shape[0])
for i in rows:
  title1 = df.iloc[i,0]
  for j in rows:
    title2 = df.iloc[j,0]
    score = computeSimilarityScore(title1, title2)
    aux = pd.DataFrame([[title1, title2, score]],
                       columns=['TITLE_1', 'TITLE_2', 'SIMILARITY_SCORE'])
    sim_df = pd.concat([sim_df,aux])

sim_df.shape

In [6]:
sim_df.shape

(0, 3)

In [ ]:
# Obtaining the dataset 1
data1 = pd.DataFrame({'P': [1,3,5]})
  
# Obtaining the dataset 2
#data2 = pd.DataFrame({'Q': [2,4,6]})
data2 = pd.DataFrame({'Q': [1,2,3]})
  
# Doing cartesian product of datasets 1 and 2 
data3 = pd.merge(data1.assign(key=1), data2.assign(key=1), 
                 on='key').drop('key', axis=1)
  
# Printing the cartesian product of both datasets
print(data3)

   P  Q
0  1  1
1  1  2
2  1  3
3  3  1
4  3  2
5  3  3
6  5  1
7  5  2
8  5  3


In [ ]:
df2 = pd.DataFrame(data=[['sravan', 'Sudheer'], 
                        ['radha', 'vani'], ],
                  columns=['gents', 'ladies'])
df3 = list(product(df['ITE_ITEM_TITLE'], df['ITE_ITEM_TITLE']))
len(df3)

In [18]:
import pandas as pd
# Create pandas Series
courses = pd.Series(["Spark","PySpark","Hadoop"])
fees = pd.Series([22000,25000])
discount  = pd.Series([1000,2300,1000])

# Combine two series.
df=pd.concat([courses,fees],axis=1)

In [1]:
sim_df = pd.DataFrame(columns=['TITLE_1', 'TITLE_2'])
rows = range(df.shape[0])
for i in rows:
  aux = pd.DataFrame()
  aux ['TITLE_2'] = df['ITE_ITEM_TITLE']
  aux ['TITLE_1'] = df.iloc[i,0]
  aux = aux[['TITLE_1','TITLE_2']]
  sim_df = pd.concat([sim_df,aux])

sim_df.shape

NameError: ignored

In [39]:
sim_df.head()

,TITLE_1,TITLE_2
